In [1]:
import numpy as np

In [2]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

In [17]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [18]:
class Sigmoid:
    def __init__(self, x):
        self.out =None

    def forward(self, x):
        self.out = sigmoid(x)
        return self.out
    
    def backward(self, dout):
        dx = dout * self.out * (1-self.out)
        return dx
        

In [19]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b        
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)
        return dx

In [20]:
class SoftmaxWithLoss:
    def __init__(self,):
        self.loss = None
        self.y = None # 
        self.t = None 

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: #
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

### 神经网络的学习总图
+ Prepreparation:权重W, 偏权重b, 
>**Steps**:
 1. 小批次: from training data, 随机取出资料
 2. 计算梯度: 计算与各权重参数有关的损失函数梯度gradient, 这里使用误差反向传播法
 3. 更新参数: 往梯度方向微量更新权重参数
 4. 重复:  重复step1, step2, step3

#### 

In [21]:
from collections import OrderedDict
## 双层NN的类别
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #权重初始化
        self.params= {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        ## 产生各层
        self.layers = OrderedDict() #有序字典:可以记录新增之字典内的元素顺序
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] =Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x) # forward的传入参数都是x,上一层的forward result, 会变成下一层的x
        
        return x
    
    def loss(self, x, t): # correct label
        # 输入data: x, t: 训练资料
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, y, t):
        y =self.predict(x)
        y = np.argmax(y, axis = 1) # each row's index of max value
        t = np.argmax(t, axis = 1) # each one-hot code: which index == 1
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x,t)
        
        #backward
        dout =1
        dout =self.lastLayer.backward(dout) #反向传播也只能依照相反的方向来call个层
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers():
             dout= layer.backward(dout) # backward的传入参数都是x,上一层的forward result, 会变成下一层的x
        
        #设定
        grads={}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Affine2'].db
        
        return grads

        

In [22]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [23]:
## 执行小批次学习
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train),(x_test, t_test) = load_mnist(normalize =True, one_hot_label=True)
train_loss_list =[]
train_acc_list = []
test_acc_list = []

##hyperparameters
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
iter_per_epoch = max(train_size / batch_size, 1)

network_100 = TwoLayerNet(input_size = 784, hidden_size=50, output_size= 10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #利用误差反向传播法计算梯度
    grads = network_100.gradient(x_batch, t_batch)
    
    #更新参数
    for key in ('W1','b1','W2','b2'):
        network_100.params[key] -= learning_rate *grads[key]
        
    #记录学习过程
    loss = network_100.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    #计算 1 epoch 的辨识准确度
    if i % iter_per_epoch == 0:
        train_acc = network_100.accuracy(x_train, t_train)
        test_acc = network_100.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train acc, test acc | '+ str(train_acc)+ ', '+str(test_acc))

TypeError: unorderable types: NoneType() <= int()